In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.headless = True

import numpy as np
import pandas as pd
import time as time
from time import sleep
import random
from tqdm import tqdm


In [2]:
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.library.parameters import SeasonAll

In [4]:
seasons = ['200{}-0{}'.format(x, x+1) if x != 9 else '200{}-{}'.format(x, x+1) for x in range(6, 10)]
seasons2 = ['20{}-{}'.format(x, x+1) for x in range(10, 21)]

seasons.extend(seasons2)
dates = []
for season in seasons:
    date = []
    for season_type in ['Regular Season', 'Playoffs']:
        games = leaguegamelog.LeagueGameLog(season=season, season_type_all_star=season_type).get_data_frames()[0]
        dates.extend(games['GAME_DATE'].unique())
    sleep(2)

dates[:10]

['2006-10-31',
 '2006-11-01',
 '2006-11-02',
 '2006-11-03',
 '2006-11-04',
 '2006-11-05',
 '2006-11-06',
 '2006-11-07',
 '2006-11-08',
 '2006-11-09']

In [3]:
dates = []
for season in ['2020-21']:
    date = []
    for season_type in ['Regular Season', 'Playoffs']:
        games = leaguegamelog.LeagueGameLog(season=season, season_type_all_star=season_type).get_data_frames()[0]
        dates.extend(games['GAME_DATE'].unique())
    sleep(2)


In [4]:
dates

['2020-12-22',
 '2020-12-23',
 '2020-12-25',
 '2020-12-26',
 '2020-12-27',
 '2020-12-28',
 '2020-12-29',
 '2020-12-30',
 '2020-12-31',
 '2021-01-01',
 '2021-01-02',
 '2021-01-03',
 '2021-01-04',
 '2021-01-05',
 '2021-01-06',
 '2021-01-07',
 '2021-01-08',
 '2021-01-09',
 '2021-01-10',
 '2021-01-11',
 '2021-01-12',
 '2021-01-13',
 '2021-01-14',
 '2021-01-15',
 '2021-01-16',
 '2021-01-17',
 '2021-01-18',
 '2021-01-19',
 '2021-01-20',
 '2021-01-21',
 '2021-01-22',
 '2021-01-23',
 '2021-01-24',
 '2021-01-25',
 '2021-01-26',
 '2021-01-27',
 '2021-01-28',
 '2021-01-29',
 '2021-01-30',
 '2021-01-31',
 '2021-02-01',
 '2021-02-02',
 '2021-02-03',
 '2021-02-04',
 '2021-02-05',
 '2021-02-06',
 '2021-02-07',
 '2021-02-08',
 '2021-02-09',
 '2021-02-10',
 '2021-02-11',
 '2021-02-12',
 '2021-02-13',
 '2021-02-14',
 '2021-02-15',
 '2021-02-16',
 '2021-02-17',
 '2021-02-18',
 '2021-02-19',
 '2021-02-20',
 '2021-02-21',
 '2021-02-22',
 '2021-02-23',
 '2021-02-24',
 '2021-02-25',
 '2021-02-26',
 '2021-02-

In [8]:
current_dates = pd.read_csv('../data/all_moneylines.csv')
current_dates = current_dates['game_date'].unique()

dates_to_add = set(dates) - set(current_dates)

print(len(dates_to_add))
print(list(dates_to_add)[:20])


81
['2021-05-23', '2021-04-01', '2021-04-03', '2021-03-27', '2021-03-31', '2021-05-08', '2021-03-18', '2021-03-26', '2021-05-01', '2021-04-28', '2021-04-20', '2021-03-21', '2021-05-11', '2021-04-16', '2021-04-09', '2021-04-26', '2021-03-30', '2021-04-04', '2021-05-07', '2021-03-28']


In [13]:
# Get Moneylines

gm_date1 = []
away_teams1 = []
home_teams1 = []
away_scoreboards1 = []
home_scoreboards1 = []
away_moneylines1 = []
home_moneylines1 = []

for date in tqdm(dates_to_add, desc='progress'):
    web = 'https://www.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date={}'.format(date)
    path = '../chromedriver.exe'
    driver = webdriver.Chrome(path)
    driver.get(web)

    single_row_events = driver.find_elements_by_class_name('eventMarketGridContainer-3QipG')

    num_postponed_events = len(driver.find_elements_by_class_name('eventStatus-3EHqw'))

    num_listed_events = len(single_row_events)
    cutoff = num_listed_events - num_postponed_events

    for event in single_row_events[:cutoff]:

        away_team = event.find_elements_by_class_name('participantBox-3ar9Y')[0].text
        home_team = event.find_elements_by_class_name('participantBox-3ar9Y')[1].text
        away_teams1.append(away_team)
        home_teams1.append(home_team)
        gm_date1.append(date)

#         scoreboard = event.find_elements_by_class_name('scoreboard-1TXQV')

#         home_score = []
#         away_score = []

#         for score in scoreboard:
#             quarters = score.find_elements_by_class_name('scoreboardColumn-2OtpR')
#             for i in range(len(quarters)):
#                 scores = quarters[i].text.split('\n')
#                 away_score.append(scores[0])
#                 home_score.append(scores[1])
#             away_scoreboards1.append(away_score)
#             home_scoreboards1.append(home_score)

        mls = event.find_elements_by_class_name('pointer-2j4Dk')
        away_lines = []
        home_lines = []
        for i in range(len(mls)):    
            if i % 2 == 0:
                away_lines.append(mls[i].text)
            else:
                home_lines.append(mls[i].text)
        away_moneylines1.append(away_lines)
        home_moneylines1.append(home_lines)

    driver.quit()
    sleep(random.randint(1,2))



progress: 100%|████████████████████████████████████████████████████████████████████████| 81/81 [12:14<00:00,  9.07s/it]


In [12]:
df1 = pd.read_csv('../data/all_moneylines_sbr.csv')
df1

,away_team,home_team,game_date,away_moneyline,home_moneyline
0,Chicago,Miami,2006-10-31,"'+180', '-', '+180', '-'","'-220', '-', '-210', '-'"
1,Phoenix,L.A. Lakers,2006-10-31,"'-380', '-', '-350', '-'","'+290', '-', '+290', '-'"
2,L.A. Clippers,Phoenix,2006-11-01,"'+165', '-', '+155', '-'","'-190', '-', '-175', '-'"
3,L.A. Lakers,Golden State,2006-11-01,"'+245', '-', '+250', '-'","'-290', '-', '-300', '-'"
4,New York,Memphis,2006-11-01,"'-', '-', '+185', '-'","'-', '-', '-215', '-'"
...,...,...,...,...,...
18437,Minnesota,Memphis,2021-04-02,"'+230', '+205', '+234', '+215'","'-280', '-245', '-285', '-265'"
18438,Atlanta,New Orleans,2021-04-02,"'-125', '-102', '-117', '-115'","'+105', '-118', '-102', '-105'"
18439,Chicago,Utah,2021-04-02,"'+475', '+660', '+506', '+505'","'-750', '-1000', '-700', '-655'"
18440,Oklahoma City,Phoenix,2021-04-02,"'+725', '+700', '+797', '+780'","'-1300', '-1100', '-1300', '-1080'"


In [19]:
print(len(away_teams1))
print(len(home_teams1))
print(len(gm_date1))
print(len(away_scoreboards1))
print(len(home_scoreboards1))
print(len(away_moneylines1))
print(len(home_moneylines1))

df1 = pd.read_csv('../data/all_moneylines_sbr.csv')
# df1
df2 = pd.DataFrame({'away_team':away_teams1,
                  'home_team':home_teams1,
                   'game_date':gm_date1,
                  'away_moneyline':away_moneylines1,
                  'home_moneyline':home_moneylines1})


updated_mls = pd.concat([df1, df2])
updated_mls = updated_mls.drop_duplicates(subset=['away_team', 'home_team', 'game_date'], keep='last')
updated_mls['game_date'] = pd.to_datetime(updated_mls['game_date'])
updated_mls.sort_values(['game_date'], inplace=True)
updated_mls
updated_mls.to_csv("../data/all_moneylines_sbr.csv", index=False)


609
609
609
0
0
609
609


In [104]:
df2

,away_team,home_team,game_date,away_moneyline,home_moneyline
0,L.A. Lakers,Charlotte,2011-02-13,"[-225, -220, -215, -]","[+185, +180, +185, -]"
1,Sacramento,Oklahoma City,2011-02-13,"[+375, +350, +400, -]","[-550, -500, -500, -]"
2,Miami,Indiana,2011-02-13,"[-240, -230, -245, -]","[+200, +190, +205, -]"
3,Charlotte,Chicago,2011-02-13,"[+400, +400, +450, -]","[-600, -600, -550, -]"
4,Atlanta,Detroit,2011-02-13,"[-, -175, -190, -]","[-, +155, +160, -]"
...,...,...,...,...,...
23853,Atlanta,Miami,2021-02-28,"[+120, +120, +125, +120]","[-140, -140, -145, -140]"
23854,Memphis,Houston,2021-02-28,"[-165, -170, -163, -165]","[+145, +150, +142, +145]"
23855,Golden State,L.A. Lakers,2021-02-28,"[-220, +125, +122, +125]","[+160, -145, -143, -145]"
23856,Phoenix,Minnesota,2021-02-28,"[-600, -530, -594, -560]","[+400, +425, +443, +435]"


In [89]:
df1 = df1.drop(columns=['away_scoreboard', 'home_scoreboard'])

In [91]:
all_moneylines = pd.concat([df1, df2])
all_moneylines.to_csv('../data/all_moneylines.csv', index=False)


In [94]:
set(dates) - set(all_moneylines['game_date'])

{'2011-02-12', '2011-02-15', '2011-02-16'}

In [98]:
dates[0]

'2010-10-26'

In [133]:
# Get Spreads

current_dates = pd.read_csv('../data/all_spreads_sbr.csv')
current_dates = current_dates['game_date'].unique()

dates_to_add = set(dates) - set(current_dates)

print("adding:", len(dates_to_add), "dates")

gm_date = []
away_teams = []
home_teams = []
away_scoreboards = []
home_scoreboards = []
away_spreads = []
home_spreads = []

for date in tqdm(dates_to_add, desc='progress'):
    web = 'https://www.sportsbookreview.com/betting-odds/nba-basketball/?date={}'.format(date)
    path = '../chromedriver.exe'
    driver = webdriver.Chrome(path)
    driver.get(web)

    single_row_events = driver.find_elements_by_class_name('eventMarketGridContainer-3QipG')

    num_postponed_events = len(driver.find_elements_by_class_name('eventStatus-3EHqw'))

    num_listed_events = len(single_row_events)
    cutoff = num_listed_events - num_postponed_events

    for event in single_row_events[:cutoff]:

        away_team = event.find_elements_by_class_name('participantBox-3ar9Y')[0].text
        home_team = event.find_elements_by_class_name('participantBox-3ar9Y')[1].text
        away_teams.append(away_team)
        home_teams.append(home_team)
        gm_date.append(date)


        scoreboard = event.find_elements_by_class_name('scoreboard-1TXQV')

        home_score = []
        away_score = []

        for score in scoreboard:
            quarters = score.find_elements_by_class_name('scoreboardColumn-2OtpR')
            for i in range(len(quarters)):
                scores = quarters[i].text.split('\n')
                away_score.append(scores[0])
                home_score.append(scores[1])
            away_scoreboards.append(away_score)
            home_scoreboards.append(home_score)
            
            
        if len(away_scoreboards) != len(away_teams):
            num_to_add = len(away_teams) - len(away_scoreboards)
            for i in range(num_to_add):
                away_scoreboards.append([])
                home_scoreboards.append([])
            
        
        spreads = event.find_elements_by_class_name('pointer-2j4Dk')
        away_lines = []
        home_lines = []
        for i in range(len(spreads)):    
            if i % 2 == 0:
                away_lines.append(spreads[i].text)
            else:
                home_lines.append(spreads[i].text)
        away_spreads.append(away_lines)
        home_spreads.append(home_lines)
        
        if len(away_spreads) != len(away_teams):
            num_to_add = len(away_teams) - len(away_spreads)
            for i in range(num_to_add):
                away_scoreboards.append([])
                home_scoreboards.append([])
            
        
    driver.quit()
    sleep(random.randint(1,2))

spread_df = pd.DataFrame({'away_team':away_teams,
                  'home_team':home_teams,
                   'game_date':gm_date,
                  'away_scoreboard':away_scoreboards,
                  'home_scoreboard':home_scoreboards,
                  'away_spreads':away_spreads,
                  'home_spreads':home_spreads})

spread_df


progress:   0%|                                                                                | 0/627 [00:00<?, ?it/s]

adding: 627 dates


progress: 100%|████████████████████████████████████████████████████████████████████| 627/627 [2:27:34<00:00, 14.12s/it]


,away_team,home_team,game_date,away_scoreboard,home_scoreboard,away_moneyline,home_moneyline
0,San Antonio,Houston,2017-05-11,"[31, 30, 26, 27, 114]","[24, 18, 22, 11, 75]","[+9-115, +9-108, +8½-110, +9-105]","[-9-105, -9-112, -8½-110, -9-105]"
1,Atlanta,Boston,2008-05-04,"[16, 10, 17, 22, 65]","[27, 17, 35, 20, 99]","[+14½-105, +15-110, +15-110, -]","[-14½-115, -15-110, -15-110, -]"
2,Utah,L.A. Lakers,2008-05-04,"[24, 17, 31, 26, 98]","[25, 29, 25, 30, 109]","[+8½-110, +8½-105, +8½-110, -]","[-8½-110, -8½-115, -8½-110, -]"
3,L.A. Lakers,Houston,2009-05-10,"[16, 20, 18, 33, 87]","[29, 25, 29, 16, 99]",[],[]
4,Boston,Orlando,2009-05-10,"[25, 23, 31, 16, 95]","[28, 18, 25, 23, 94]","[+5-115, +5½-110, +5½-110, -]","[-5-105, -5½-110, -5½-110, -]"
...,...,...,...,...,...,...,...
1167,Phoenix,San Antonio,2007-05-12,"[30, 23, 19, 29, 101]","[25, 30, 25, 28, 108]","[+4-110, +4½-112, +4-110, -]","[-4-110, -4½-108, -4-110, -]"
1168,Toronto,Orlando,2008-04-28,"[26, 24, 18, 24, 92]","[22, 27, 25, 28, 102]","[+7½-105, +7½-110, +7-110, -]","[-7½-115, -7½-110, -7-110, -]"
1169,Boston,Atlanta,2008-04-28,"[24, 24, 27, 17, 92]","[29, 22, 14, 32, 97]","[-10-105, -9-110, -9-110, -]","[+10-115, +9-110, +9-110, -]"
1170,L.A. Lakers,Denver,2008-04-28,"[32, 32, 15, 28, 107]","[23, 31, 23, 24, 101]","[-5-105, -4-110, -4-110, -]","[+5-115, +4-110, +4-110, -]"


In [135]:
spreads_orig = pd.read_csv("../data/all_spreads_sbr.csv")

updated_spreads = pd.concat([spreads_orig, spread_df])

updated_spreads['game_date'] = pd.to_datetime(updated_spreads['game_date'])
updated_spreads.sort_values(['game_date'], inplace=True)

updated_spreads.rename(columns={'away_moneyline':'away_spread',
                               'home_moneyline':'home_spread'}, inplace=True)
updated_spreads

,away_team,home_team,game_date,away_scoreboard,home_scoreboard,away_spread,home_spread
0,Chicago,Miami,2006-10-31,"['22', '37', '21', '28', '108']","['16', '14', '21', '15', '66']","['+4½-110', '-', '+4½-110', '-']","['-4½-110', '-', '-4½-110', '-']"
1129,Phoenix,L.A. Lakers,2006-10-31,"['41', '17', '21', '27', '106']","['26', '27', '34', '27', '114']","['-8½-110', '-', '-8-110', '-']","['+8½-110', '-', '+8-110', '-']"
1128,Chicago,Miami,2006-10-31,"['22', '37', '21', '28', '108']","['16', '14', '21', '15', '66']","['+4½-110', '-', '+4½-110', '-']","['-4½-110', '-', '-4½-110', '-']"
1,Phoenix,L.A. Lakers,2006-10-31,"['41', '17', '21', '27', '106']","['26', '27', '34', '27', '114']","['-8½-110', '-', '-8-110', '-']","['+8½-110', '-', '+8-110', '-']"
3,Chicago,Orlando,2006-11-01,"['20', '25', '25', '24', '94']","['32', '31', '23', '23', '109']","['-2-110', '-', '-1-110', '-']","['+2-110', '-', '+1-110', '-']"
...,...,...,...,...,...,...,...
501,Atlanta,Orlando,2021-03-03,"[27, 15, 36, 37, 115]","[39, 22, 29, 22, 112]","[-3-105, -3-107, -2½-110, -2½-108]","[+3-115, +3-113, +2½-110, +2½-108]"
500,Brooklyn,Houston,2021-03-03,"[30, 37, 34, 31, 132]","[27, 27, 31, 29, 114]","[-10½-115, -10½-110, -11-110, -11-108]","[+10½-105, +10½-110, +11-110, +11-108]"
499,Detroit,Toronto,2021-03-03,"[43, 26, 29, 31, 129]","[37, 23, 20, 25, 105]","[+7½-110, +7½-110, +7½-105, +8-108]","[-7½-110, -7½-110, -7½-115, -8-108]"
506,Golden State,Portland,2021-03-03,"[29, 27, 24, 26, 106]","[28, 27, 27, 26, 108]","[-5½-120, -1-115, -1-115, -1-108]","[+5½-120, +1-105, +1-105, +1-108]"


In [138]:
updated_spreads = updated_spreads.drop_duplicates(['away_team','home_team', 'game_date'])

In [139]:
updated_spreads.to_csv("../data/all_spreads_sbr.csv", index=False)

In [114]:
spread_df

,away_team,home_team,game_date,away_scoreboard,home_scoreboard,away_moneyline,home_moneyline
0,Chicago,Miami,2006-10-31,"[22, 37, 21, 28, 108]","[16, 14, 21, 15, 66]","[+4½-110, -, +4½-110, -]","[-4½-110, -, -4½-110, -]"
1,Phoenix,L.A. Lakers,2006-10-31,"[41, 17, 21, 27, 106]","[26, 27, 34, 27, 114]","[-8½-110, -, -8-110, -]","[+8½-110, -, +8-110, -]"
2,Indiana,Charlotte,2006-11-01,"[23, 26, 28, 29, 106]","[27, 23, 23, 26, 99]","[-1-110, -, -1-110, -]","[+1-110, -, +1-110, -]"
3,Chicago,Orlando,2006-11-01,"[20, 25, 25, 24, 94]","[32, 31, 23, 23, 109]","[-2-110, -, -1-110, -]","[+2-110, -, +1-110, -]"
4,Atlanta,Philadelphia,2006-11-01,"[14, 21, 23, 17, 75]","[29, 20, 24, 15, 88]","[+6-115, -, +5½-110, -]","[-6-105, -, -5½-110, -]"
...,...,...,...,...,...,...,...
33579,Atlanta,Miami,2021-03-02,"[23, 21, 19, 31, 94]","[17, 20, 29, 14, 80]","[+2-110, +2½-115, +2½-110, +2-108]","[-2-110, -2½-105, -2½-110, -2-108]"
33580,L.A. Clippers,Boston,2021-03-02,"[32, 31, 26, 23, 112]","[35, 27, 26, 29, 117]","[+8½-120, -5-110, -3-110, -5-108]","[-8½-120, +5-110, +3-110, +5-108]"
33581,New York,San Antonio,2021-03-02,"[23, 24, 21, 25, 93]","[25, 26, 36, 32, 119]","[-2-105, -1½-113, -1½-105, -2-108]","[+2-115, +1½-107, +1½-115, +2-108]"
33582,Denver,Milwaukee,2021-03-02,"[37, 27, 35, 29, 128]","[23, 27, 30, 17, 97]","[+8-110, +8-109, +7½-105, +8-108]","[-8-110, -8-111, -7½-115, -8-108]"


In [115]:
spread_df.rename(columns={'away_moneyline':'away_spread', 'home_moneyline':'home_spread'})
spread_df.to_csv("../data/all_spreads_sbr.csv", index=False)

In [82]:

gm_date = []
away_teams = []
home_teams = []
away_scoreboards = []
home_scoreboards = []
away_moneylines = []
home_moneylines = []

web = 'https://www.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date={}'.format(date)
path = '../chromedriver.exe'
driver = webdriver.Chrome(path)
driver.get(web)

single_row_events = driver.find_elements_by_class_name('eventMarketGridContainer-3QipG')

num_postponed_events = len(driver.find_elements_by_class_name('eventStatus-3EHqw'))

num_listed_events = len(single_row_events)
cutoff = num_listed_events - num_postponed_events

for event in single_row_events[:cutoff]:

    away_team = event.find_elements_by_class_name('participantBox-3ar9Y')[0].text
    home_team = event.find_elements_by_class_name('participantBox-3ar9Y')[1].text
    away_teams.append(away_team)
    home_teams.append(home_team)
    gm_date.append(date)


    scoreboard = event.find_elements_by_class_name('scoreboard-1TXQV')

    home_score = []
    away_score = []

    for score in scoreboard:
        quarters = score.find_elements_by_class_name('scoreboardColumn-2OtpR')
        for i in range(len(quarters)):
            scores = quarters[i].text.split('\n')
            away_score.append(scores[0])
            home_score.append(scores[1])

        away_scoreboards.append(away_score)
        home_scoreboards.append(home_score)

    mls = event.find_elements_by_class_name('pointer-2j4Dk')
    away_lines = []
    home_lines = []
    for i in range(len(mls)):    
        if i % 2 == 0:
            away_lines.append(mls[i].text)
        else:
            home_lines.append(mls[i].text)
    away_moneylines.append(away_lines)
    home_moneylines.append(home_lines)

    driver.quit()

### 1st Half Spreads

In [4]:

seasons = ['200{}-0{}'.format(x, x+1) if x!=9 else '2009-10' for x in range(6, 10)]
seasons2 = ['20{}-{}'.format(x, x+1) for x in range(10, 20)]
seasons.extend(seasons2)
seasons
dates = []
for season in seasons:
    df = pd.read_csv("../data/basic_team_boxscores/team_gamelogs_{}.csv".format(season))
    season_dates = df['GAME_DATE'].unique()
    dates.extend(season_dates)
    
dates[:10]

['2006-10-31',
 '2006-11-01',
 '2006-11-02',
 '2006-11-03',
 '2006-11-04',
 '2006-11-05',
 '2006-11-06',
 '2006-11-07',
 '2006-11-08',
 '2006-11-09']

In [10]:
dates = []
for season_type in ['Regular Season', 'Playoffs']:
    gamelog = leaguegamelog.LeagueGameLog(season_type_all_star=season_type).get_data_frames()[0]    
    dates.extend(gamelog['GAME_DATE'].tolist())
    
dates[-1]
    
    

'2021-06-07'

In [5]:
current_spreads_1H = pd.read_csv("../data/spreads_1H.csv")
missing_dates = set([])
# spread_df.loc[spread_df['away_1H_spread'] == "-, -, -, -"]
# spread_df.loc[spread_df['home_1H_spread'] == "-, -, -, -"]
dates = current_spreads_1H.loc[current_spreads_1H['home_1H_spread'].isnull(), 'game_date'].astype(str)
missing_dates.update(dates)
# spread_df.loc[274:274]
len(missing_dates)

missing_dates

set()

In [11]:
current_dates = current_spreads_1H.loc[current_spreads_1H['home_1H_spread'].isnull(), 'game_date'].astype(str)
set(dates) - set(current_dates) 

{'2020-12-22',
 '2020-12-23',
 '2020-12-25',
 '2020-12-26',
 '2020-12-27',
 '2020-12-28',
 '2020-12-29',
 '2020-12-30',
 '2020-12-31',
 '2021-01-01',
 '2021-01-02',
 '2021-01-03',
 '2021-01-04',
 '2021-01-05',
 '2021-01-06',
 '2021-01-07',
 '2021-01-08',
 '2021-01-09',
 '2021-01-10',
 '2021-01-11',
 '2021-01-12',
 '2021-01-13',
 '2021-01-14',
 '2021-01-15',
 '2021-01-16',
 '2021-01-17',
 '2021-01-18',
 '2021-01-19',
 '2021-01-20',
 '2021-01-21',
 '2021-01-22',
 '2021-01-23',
 '2021-01-24',
 '2021-01-25',
 '2021-01-26',
 '2021-01-27',
 '2021-01-28',
 '2021-01-29',
 '2021-01-30',
 '2021-01-31',
 '2021-02-01',
 '2021-02-02',
 '2021-02-03',
 '2021-02-04',
 '2021-02-05',
 '2021-02-06',
 '2021-02-07',
 '2021-02-08',
 '2021-02-09',
 '2021-02-10',
 '2021-02-11',
 '2021-02-12',
 '2021-02-13',
 '2021-02-14',
 '2021-02-15',
 '2021-02-16',
 '2021-02-17',
 '2021-02-18',
 '2021-02-19',
 '2021-02-20',
 '2021-02-21',
 '2021-02-22',
 '2021-02-23',
 '2021-02-24',
 '2021-02-25',
 '2021-02-26',
 '2021-02-

In [31]:

gm_date = []
away_teams = []
home_teams = []
away_spreads = []
home_spreads = []


for date in tqdm(missing_dates, desc='progress'):
    web = 'https://www.sportsbookreview.com/betting-odds/nba-basketball/pointspread/1st-half/?date={}'.format(date)
    path = '../chromedriver.exe'
    driver = webdriver.Chrome(path)
    driver.get(web)
    
    sleep(random.randint(4,5))
    single_row_events = driver.find_elements_by_class_name('eventMarketGridContainer-3QipG')

    num_postponed_events = len(driver.find_elements_by_class_name('eventStatus-3EHqw'))

    num_listed_events = len(single_row_events)
    cutoff = num_listed_events - num_postponed_events

    for event in single_row_events[:cutoff]:

        away_team = event.find_elements_by_class_name('participantBox-3ar9Y')[0].text
        home_team = event.find_elements_by_class_name('participantBox-3ar9Y')[1].text
        away_teams.append(away_team)
        home_teams.append(home_team)
        gm_date.append(date)


#         scoreboard = event.find_elements_by_class_name('scoreboard-1TXQV')

#         home_score = []
#         away_score = []

#         for score in scoreboard:
#             quarters = score.find_elements_by_class_name('scoreboardColumn-2OtpR')
#             for i in range(len(quarters)):
#                 scores = quarters[i].text.split('\n')
#                 away_score.append(scores[0])
#                 home_score.append(scores[1])
#             away_scoreboards.append(away_score)
#             home_scoreboards.append(home_score)
            
            
#         if len(away_scoreboards) != len(away_teams):
#             num_to_add = len(away_teams) - len(away_scoreboards)
#             for i in range(num_to_add):
#                 away_scoreboards.append([])
#                 home_scoreboards.append([])
            
        
        spreads = event.find_elements_by_class_name('pointer-2j4Dk')
        away_lines = []
        home_lines = []
        for i in range(len(spreads)):    
            if i % 2 == 0:
                away_lines.append(spreads[i].text)
            else:
                home_lines.append(spreads[i].text)
        away_spreads.append(away_lines)
        home_spreads.append(home_lines)
        
        if len(away_spreads) != len(away_teams):
            num_to_add = len(away_teams) - len(away_spreads)
            for i in range(num_to_add):
                away_spreads.append([])
                home_spreads.append([])
            
        
    driver.quit()

spreads_1H_df = pd.DataFrame({'away_team':away_teams,
                  'home_team':home_teams,
                   'game_date':gm_date,
                  'away_1H_spread':away_spreads,
                  'home_1H_spread':home_spreads})

spreads_1H_df


progress: 100%|██████████████████████████████████████████████████████████████████████████| 5/5 [01:10<00:00, 14.01s/it]


,away_team,home_team,game_date,away_1H_spread,home_1H_spread
0,Utah,Indiana,2019-11-27,"[PK-105, PK+100, +½-110, PK-105]","[PK-115, PK-120, -½-110, PK-115]"
1,Brooklyn,Boston,2019-11-27,"[+4½-105, +4½-115, +4½-115, +4½-110]","[-4½-115, -4½-105, -4½-105, -4½-110]"
2,Sacramento,Philadelphia,2019-11-27,"[+5½-110, +5½-110, +5½-105, +5½-110]","[-5½-110, -5½-110, -5½-115, -5½-110]"
3,Orlando,Cleveland,2019-11-27,"[PK-105, PK+100, +½-120, PK-105]","[PK-115, PK-120, -½+100, PK-115]"
4,Detroit,Charlotte,2019-11-27,"[-1½-115, -1½-110, -1½-105, -1½-110]","[+1½-105, +1½-110, +1½-115, +1½-110]"
5,New York,Toronto,2019-11-27,"[+6-110, +6-115, +5½-115, +6-115]","[-6-110, -6-105, -5½-105, -6-105]"
6,Atlanta,Milwaukee,2019-11-27,"[+8½-110, +8½-110, +8½-110, +8½-110]","[-8½-110, -8½-110, -8½-110, -8½-110]"
7,L.A. Clippers,Memphis,2019-11-27,"[-3-110, -3-101, -2½-110, -3-105]","[+3-110, +3-119, +2½-110, +3-115]"
8,Miami,Houston,2019-11-27,"[+4½-110, +4-105, +4½-110, +4-105]","[-4½-110, -4-115, -4½-110, -4-115]"
9,Minnesota,San Antonio,2019-11-27,"[+1½-110, +1½-110, +1½-110, +1½-105]","[-1½-110, -1½-110, -1½-110, -1½-115]"


In [32]:
current_spreads_1H

,away_team,home_team,game_date,away_1H_spread,home_1H_spread
0,Chicago,Miami,2006-10-31,"'-', '-', '+2½-110', '-'","'-', '-', '-2½-110', '-'"
1,Phoenix,L.A. Lakers,2006-10-31,"'-', '-', '-4½-115', '-'","'-', '-', '+4½-105', '-'"
2,Indiana,Charlotte,2006-11-01,"'-', '-', 'PK-110', '-'","'-', '-', 'PK-110', '-'"
3,Chicago,Orlando,2006-11-01,"'-', '-', 'PK-120', '-'","'-', '-', 'PK+100', '-'"
4,Atlanta,Philadelphia,2006-11-01,"'-', '-', '+2½-105', '-'","'-', '-', '-2½-115', '-'"
...,...,...,...,...,...
17702,Utah,Memphis,2019-11-29,NaN,NaN
17703,L.A. Clippers,San Antonio,2019-11-29,NaN,NaN
17704,Dallas,Phoenix,2019-11-29,NaN,NaN
17705,Chicago,Portland,2019-11-29,NaN,NaN


In [33]:
spreads_1H_df
for col in spreads_1H_df.columns[3:]:
    spreads_1H_df[col] = spreads_1H_df[col].astype(str)
    spreads_1H_df[col] = spreads_1H_df[col].str.replace("[", "")
    spreads_1H_df[col] = spreads_1H_df[col].str.replace("]", "")
    spreads_1H_df[col] = spreads_1H_df[col].str.strip()
    
spreads_1H_df

,away_team,home_team,game_date,away_1H_spread,home_1H_spread
0,Utah,Indiana,2019-11-27,"'PK-105', 'PK+100', '+½-110', 'PK-105'","'PK-115', 'PK-120', '-½-110', 'PK-115'"
1,Brooklyn,Boston,2019-11-27,"'+4½-105', '+4½-115', '+4½-115', '+4½-110'","'-4½-115', '-4½-105', '-4½-105', '-4½-110'"
2,Sacramento,Philadelphia,2019-11-27,"'+5½-110', '+5½-110', '+5½-105', '+5½-110'","'-5½-110', '-5½-110', '-5½-115', '-5½-110'"
3,Orlando,Cleveland,2019-11-27,"'PK-105', 'PK+100', '+½-120', 'PK-105'","'PK-115', 'PK-120', '-½+100', 'PK-115'"
4,Detroit,Charlotte,2019-11-27,"'-1½-115', '-1½-110', '-1½-105', '-1½-110'","'+1½-105', '+1½-110', '+1½-115', '+1½-110'"
5,New York,Toronto,2019-11-27,"'+6-110', '+6-115', '+5½-115', '+6-115'","'-6-110', '-6-105', '-5½-105', '-6-105'"
6,Atlanta,Milwaukee,2019-11-27,"'+8½-110', '+8½-110', '+8½-110', '+8½-110'","'-8½-110', '-8½-110', '-8½-110', '-8½-110'"
7,L.A. Clippers,Memphis,2019-11-27,"'-3-110', '-3-101', '-2½-110', '-3-105'","'+3-110', '+3-119', '+2½-110', '+3-115'"
8,Miami,Houston,2019-11-27,"'+4½-110', '+4-105', '+4½-110', '+4-105'","'-4½-110', '-4-115', '-4½-110', '-4-115'"
9,Minnesota,San Antonio,2019-11-27,"'+1½-110', '+1½-110', '+1½-110', '+1½-105'","'-1½-110', '-1½-110', '-1½-110', '-1½-115'"


In [34]:
current_spreads_1H = pd.concat([current_spreads_1H, spreads_1H_df])
current_spreads_1H = current_spreads_1H.drop_duplicates(subset = ['away_team', 'home_team', 'game_date'], keep='last')

In [35]:
spreads_1H_df['game_date'] = pd.to_datetime(spreads_1H_df['game_date'])
spreads_1H_df.sort_values('game_date', inplace=True)
current_spreads_1H.to_csv('../data/spreads_1H.csv', index=False)


In [36]:
current_spreads_1H

,away_team,home_team,game_date,away_1H_spread,home_1H_spread
0,Chicago,Miami,2006-10-31,"'-', '-', '+2½-110', '-'","'-', '-', '-2½-110', '-'"
1,Phoenix,L.A. Lakers,2006-10-31,"'-', '-', '-4½-115', '-'","'-', '-', '+4½-105', '-'"
2,Indiana,Charlotte,2006-11-01,"'-', '-', 'PK-110', '-'","'-', '-', 'PK-110', '-'"
3,Chicago,Orlando,2006-11-01,"'-', '-', 'PK-120', '-'","'-', '-', 'PK+100', '-'"
4,Atlanta,Philadelphia,2006-11-01,"'-', '-', '+2½-105', '-'","'-', '-', '-2½-115', '-'"
...,...,...,...,...,...
55,Utah,Memphis,2019-11-29,"'-3½-115', '-3-120', '-4-105', '-3½-115'","'+3½-105', '+3+100', '+4-115', '+3½-105'"
56,L.A. Clippers,San Antonio,2019-11-29,"'-2½-110', '-2½-110', '-2½-115', '-2½-115'","'+2½-110', '+2½-110', '+2½-105', '+2½-105'"
57,Dallas,Phoenix,2019-11-29,"'-½-110', 'PK-125', '-½-115', '-1-105'","'+½-110', 'PK+105', '+½-105', '+1-115'"
58,Chicago,Portland,2019-11-29,"'+4-115', '+3½-103', '+3½-105', '+4-110'","'-4-105', '-3½-117', '-3½-115', '-4-110'"


In [18]:
spreads_1H_df
for col in spreads_1H_df.columns[3:]:
    spreads_1H_df[col] = spreads_1H_df[col].astype(str)
    spreads_1H_df[col] = spreads_1H_df[col].str.replace("[", "")
    spreads_1H_df[col] = spreads_1H_df[col].str.replace("]", "")
    spreads_1H_df[col] = spreads_1H_df[col].str.strip()
    
spreads_1H_df['game_date'] = pd.to_datetime(spreads_1H_df['game_date'])
spreads_1H_df.sort_values('game_date', inplace=True)
spreads_1H_df

,away_team,home_team,game_date,away_1H_spread,home_1H_spread
0,Chicago,Miami,2006-10-31,"'-', '-', '+2½-110', '-'","'-', '-', '-2½-110', '-'"
1,Phoenix,L.A. Lakers,2006-10-31,"'-', '-', '-4½-115', '-'","'-', '-', '+4½-105', '-'"
2,Indiana,Charlotte,2006-11-01,"'-', '-', 'PK-110', '-'","'-', '-', 'PK-110', '-'"
3,Chicago,Orlando,2006-11-01,"'-', '-', 'PK-120', '-'","'-', '-', 'PK+100', '-'"
4,Atlanta,Philadelphia,2006-11-01,"'-', '-', '+2½-105', '-'","'-', '-', '-2½-115', '-'"
...,...,...,...,...,...
17702,Miami,L.A. Lakers,2020-10-02,"'+6½+100', '+6-105', '+6-103', '+6½-110'","'-6½-120', '-6-115', '-6-118', '-6½-110'"
17703,L.A. Lakers,Miami,2020-10-04,"'-6-105', '-5½-117', '-6-110', '-6-110'","'+6-115', '+5½-103', '+6-110', '+6-110'"
17704,L.A. Lakers,Miami,2020-10-06,"'-5-125', '-5-110', '-4½+111', '-5½-105'","'+5+105', '+5-110', '+4½-133', '+5½-115'"
17705,Miami,L.A. Lakers,2020-10-09,"'+4-105', '+4-105', '+3½-110', '+4½-115'","'-4-115', '-4-115', '-3½-110', '-4½-105'"


In [19]:
spreads_1H_df.to_csv("../data/first_half_spreads.csv", index=False)

### 1st Half Moneylines

In [5]:
dates[:10]

['2006-10-31',
 '2006-11-01',
 '2006-11-02',
 '2006-11-03',
 '2006-11-04',
 '2006-11-05',
 '2006-11-06',
 '2006-11-07',
 '2006-11-08',
 '2006-11-09']

In [8]:
# Get Moneylines

gm_date = []
away_teams = []
home_teams = []
away_scoreboards = []
home_scoreboards = []
away_moneylines_1H = []
home_moneylines_1H = []

for date in tqdm(dates[:2], desc='progress'):
    web = 'https://www.sportsbookreview.com/betting-odds/nba-basketball/money-line/1st-half/?date={}'.format(date)
    path = '../chromedriver.exe'
    driver = webdriver.Chrome(path)
    driver.get(web)
    sleep(random.randint(1,2))

    single_row_events = driver.find_elements_by_class_name('eventMarketGridContainer-3QipG')

    num_postponed_events = len(driver.find_elements_by_class_name('eventStatus-3EHqw'))

    num_listed_events = len(single_row_events)
    cutoff = num_listed_events - num_postponed_events

    for event in single_row_events[:cutoff]:

        away_team = event.find_elements_by_class_name('participantBox-3ar9Y')[0].text
        home_team = event.find_elements_by_class_name('participantBox-3ar9Y')[1].text
        away_teams.append(away_team)
        home_teams.append(home_team)
        gm_date.append(date)

        scoreboard = event.find_elements_by_class_name('scoreboard-1TXQV')

        home_score = []
        away_score = []

        for score in scoreboard:
            quarters = score.find_elements_by_class_name('scoreboardColumn-2OtpR')
            for i in range(len(quarters)):
                scores = quarters[i].text.split('\n')
                away_score.append(scores[0])
                home_score.append(scores[1])
            away_scoreboards.append(away_score)
            home_scoreboards.append(home_score)            
            
        if len(away_scoreboards) != len(away_teams):
            num_to_add = len(away_teams) - len(away_scoreboards)
            for i in range(num_to_add):
                away_scoreboards.append([])
                home_scoreboards.append([])

        mls = event.find_elements_by_class_name('pointer-2j4Dk')
        away_lines = []
        home_lines = []
        for i in range(len(mls)):    
            if i % 2 == 0:
                away_lines.append(mls[i].text)
            else:
                home_lines.append(mls[i].text)
        away_moneylines_1H.append(away_lines)
        home_moneylines_1H.append(home_lines)

    driver.quit()

mls_1H_df = pd.DataFrame({'away_team':away_teams,
                  'home_team':home_teams,
                   'game_date':gm_date,
                          'away_scoreboard':away_scoreboards,
                          'home_scoreboard':home_scoreboards,
                  'away_1H_ml':away_moneylines_1H,
                  'home_1H_ml':home_moneylines_1H})

mls_1H_df

progress: 100%|██████████████████████████████████████████████████████████████████████████| 2/2 [00:22<00:00, 11.41s/it]


,away_team,home_team,game_date,away_scoreboard,home_scoreboard,away_1H_ml,home_1H_ml
0,Chicago,Miami,2006-10-31,"[22, 37, 21, 28, 108]","[16, 14, 21, 15, 66]","[-, -, -, -]","[-, -, -, -]"
1,Phoenix,L.A. Lakers,2006-10-31,"[41, 17, 21, 27, 106]","[26, 27, 34, 27, 114]","[-, -, -, -]","[-, -, -, -]"
2,Indiana,Charlotte,2006-11-01,"[23, 26, 28, 29, 106]","[27, 23, 23, 26, 99]","[-, -, -, -]","[-, -, -, -]"
3,Chicago,Orlando,2006-11-01,"[20, 25, 25, 24, 94]","[32, 31, 23, 23, 109]","[-, -, -, -]","[-, -, -, -]"
4,Atlanta,Philadelphia,2006-11-01,"[14, 21, 23, 17, 75]","[29, 20, 24, 15, 88]","[-, -, -, -]","[-, -, -, -]"
5,New Orleans,Boston,2006-11-01,"[26, 28, 21, 16, 91]","[16, 25, 24, 22, 87]","[-, -, -, -]","[-, -, -, -]"
6,Toronto,Brooklyn,2006-11-01,"[27, 22, 21, 22, 92]","[28, 25, 21, 28, 102]","[-, -, -, -]","[-, -, -, -]"
7,Milwaukee,Detroit,2006-11-01,"[30, 25, 23, 27, 105]","[21, 24, 27, 25, 97]","[-, -, -, -]","[-, -, -, -]"
8,Sacramento,Minnesota,2006-11-01,"[16, 25, 25, 17, 83]","[23, 20, 21, 28, 92]","[-, -, -, -]","[-, -, -, -]"
9,New York,Memphis,2006-11-01,"[20, 31, 20, 18, 89]","[25, 21, 14, 29, 89]","[-, -, -, -]","[-, -, -, -]"


In [12]:
web = 'https://www.sportsbookreview.com/betting-odds/nba-basketball/money-line/1st-half/?date=2006-10-31'.format(date)
path = '../chromedriver.exe'
driver = webdriver.Chrome(path)
driver.get(web)

single_row_events = driver.find_elements_by_class_name('eventMarketGridContainer-3QipG')

num_postponed_events = len(driver.find_elements_by_class_name('eventStatus-3EHqw'))

num_listed_events = len(single_row_events)
cutoff = num_listed_events - num_postponed_events

for event in single_row_events[:cutoff]:
    event.find_elements_by_class_name('pointer-2j4dk')

InvalidSelectorException: Message: invalid selector: An invalid or illegal selector was specified
  (Session info: chrome=89.0.4389.114)


## Update Betting Data

### Moneylines

In [ ]:
def get_dates_to_scrape(season='2020-21'):
    dates = []
    for season_type in ['Regular Season', 'Playoffs']:
        games = leaguegamelog.LeagueGameLog(season=season, season_type_all_star=season_type).get_data_frames()[0]
        dates.extend(games['GAME_DATE'].unique())
        sleep(2)
    return dates


In [25]:
current_moneyline_df = pd.read_csv('../data/all_moneylines_sbr.csv')
missing_lines = current_moneyline_df.loc[(current_moneyline_df['away_moneyline'] == '[]') 
                         | (current_moneyline_df['home_moneyline'] == '[]')]

missing_dates = missing_lines['game_date'].unique()

current_moneyline_dates = set(current_moneyline_df['game_date'].unique().tolist())

In [27]:
dates_to_scrape_ml = set(dates)-current_moneyline_dates

In [28]:
# Get Moneylines

gm_date1 = []
away_teams1 = []
home_teams1 = []
away_moneylines1 = []
home_moneylines1 = []

for date in tqdm(dates_to_scrape_ml, desc='progress'):
    web = 'https://www.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date={}'.format(date)
    path = '../chromedriver.exe'
    driver = webdriver.Chrome(path)
    driver.get(web)
    sleep(random.randint(2,3))

    single_row_events = driver.find_elements_by_class_name('eventMarketGridContainer-3QipG')

    num_postponed_events = len(driver.find_elements_by_class_name('eventStatus-3EHqw'))

    num_listed_events = len(single_row_events)
    cutoff = num_listed_events - num_postponed_events

    for event in single_row_events[:cutoff]:

        away_team = event.find_elements_by_class_name('participantBox-3ar9Y')[0].text
        home_team = event.find_elements_by_class_name('participantBox-3ar9Y')[1].text
        away_teams1.append(away_team)
        home_teams1.append(home_team)
        gm_date1.append(date)


        mls = event.find_elements_by_class_name('pointer-2j4Dk')
        away_lines = []
        home_lines = []
        for i in range(len(mls)):    
            if i % 2 == 0:
                away_lines.append(mls[i].text)
            else:
                home_lines.append(mls[i].text)
        away_moneylines1.append(away_lines)
        home_moneylines1.append(home_lines)

    driver.quit()

moneylines_to_add_df = pd.DataFrame({'away_team':away_teams1,
             'home_team':home_teams1,
             'game_date':gm_date1,
             'away_moneyline':away_moneylines1,
             'home_moneyline':home_moneylines1})

moneylines_to_add_df

progress: 100%|████████████████████████████████████████████████████████████████████████| 16/16 [02:24<00:00,  9.01s/it]


,away_team,home_team,game_date,away_moneyline,home_moneyline
0,Denver,Portland,2021-06-03,"[+175, +192, +180, +185]","[-205, -230, -220, -220]"
1,Phoenix,L.A. Lakers,2021-06-03,"[+128, +129, +130, +135]","[-148, -149, -150, -155]"
2,L.A. Clippers,Dallas,2021-06-04,"[-155, -142, -140, -145]","[+135, +122, +120, +125]"
3,Brooklyn,Milwaukee,2021-06-13,"[-140, -140, -145, -140]","[+120, +120, +125, +120]"
4,Phoenix,Denver,2021-06-13,"[-150, -242, -165, -155]","[+130, +184, +145, +135]"
5,Philadelphia,Washington,2021-05-31,"[-380, -388, -420, -425]","[+310, +309, +310, +335]"
6,Utah,Memphis,2021-05-31,"[-235, -243, -240, -240]","[+205, +202, +200, +200]"
7,Utah,L.A. Clippers,2021-06-12,"[+170, +102, +175, +180]","[-195, -120, -210, -210]"
8,Washington,Philadelphia,2021-06-02,"[+205, +203, +195, +195]","[-235, -245, -235, -230]"
9,Atlanta,New York,2021-06-02,"[-105, -105, -105, +105]","[-115, -115, -115, -125]"


In [121]:
current_moneyline_df = pd.read_csv('../data/all_moneylines_sbr.csv')

updated_moneylines = pd.concat([current_moneyline_df, moneylines_to_add_df])

updated_moneylines = updated_moneylines.drop_duplicates(['away_team', 'home_team', 'game_date'], keep='last')
updated_moneylines['game_date'] = pd.to_datetime(updated_moneylines['game_date'])
updated_moneylines = updated_moneylines.sort_values('game_date')
updated_moneylines



,away_team,home_team,game_date,away_moneyline,home_moneyline
0,Chicago,Miami,2006-10-31,"['+180', '-', '+180', '-']","['-220', '-', '-210', '-']"
1,Phoenix,L.A. Lakers,2006-10-31,"['-380', '-', '-350', '-']","['+290', '-', '+290', '-']"
13,Sacramento,Minnesota,2006-11-01,"['+190', '-', '+180', '-']","['-230', '-', '-210', '-']"
12,Atlanta,Philadelphia,2006-11-01,"['+200', '-', '+200', '-']","['-240', '-', '-240', '-']"
11,New Orleans,Boston,2006-11-01,"['+140', '-', '+140', '-']","['-160', '-', '-160', '-']"
...,...,...,...,...,...
18239,Detroit,Toronto,2021-03-03,"['+255', '+260', '+267', '+265']","['-310', '-310', '-331', '-325']"
18240,Utah,Philadelphia,2021-03-03,"['-155', '-165', '-145', '-150']","['+135', '+145', '+125', '+130']"
18241,Indiana,Cleveland,2021-03-03,"['-270', '-215', '-250', '-260']","['+220', '+185', '+207', '+210']"
18243,L.A. Lakers,Sacramento,2021-03-03,"['+165', '+158', '+163', '+160']","['-190', '-178', '-190', '-185']"


In [122]:
for col in updated_moneylines.columns[3:]:
    updated_moneylines[col] = updated_moneylines[col].astype(str)
    updated_moneylines[col] = updated_moneylines[col].str.replace("[", "")
    updated_moneylines[col] = updated_moneylines[col].str.replace("]", "")
    updated_moneylines[col] = updated_moneylines[col].str.strip()
    
updated_moneylines

,away_team,home_team,game_date,away_moneyline,home_moneyline
0,Chicago,Miami,2006-10-31,"'+180', '-', '+180', '-'","'-220', '-', '-210', '-'"
1,Phoenix,L.A. Lakers,2006-10-31,"'-380', '-', '-350', '-'","'+290', '-', '+290', '-'"
13,Sacramento,Minnesota,2006-11-01,"'+190', '-', '+180', '-'","'-230', '-', '-210', '-'"
12,Atlanta,Philadelphia,2006-11-01,"'+200', '-', '+200', '-'","'-240', '-', '-240', '-'"
11,New Orleans,Boston,2006-11-01,"'+140', '-', '+140', '-'","'-160', '-', '-160', '-'"
...,...,...,...,...,...
18239,Detroit,Toronto,2021-03-03,"'+255', '+260', '+267', '+265'","'-310', '-310', '-331', '-325'"
18240,Utah,Philadelphia,2021-03-03,"'-155', '-165', '-145', '-150'","'+135', '+145', '+125', '+130'"
18241,Indiana,Cleveland,2021-03-03,"'-270', '-215', '-250', '-260'","'+220', '+185', '+207', '+210'"
18243,L.A. Lakers,Sacramento,2021-03-03,"'+165', '+158', '+163', '+160'","'-190', '-178', '-190', '-185'"


In [123]:
updated_moneylines.to_csv("../data/all_moneylines_sbr.csv", index=False)

### Update Spreads

In [3]:
current_spread_df = pd.read_csv('../data/all_spreads_sbr.csv')
current_spread_df.loc[(current_spread_df['away_spread'] == '') | (current_spread_df['home_spread'] == '')]

,away_team,home_team,game_date,away_scoreboard,home_scoreboard,away_spread,home_spread


In [4]:
current_spread_df = pd.read_csv('../data/all_spreads_sbr.csv')
missing_dates = current_spread_df.loc[(current_spread_df['away_spread'] == '[]') | (current_spread_df['home_spread'] == '[]'), 'game_date'].unique()
len(missing_dates)

0

In [23]:
dates = get_dates_to_scrape()

dates

['2020-12-22',
 '2020-12-23',
 '2020-12-25',
 '2020-12-26',
 '2020-12-27',
 '2020-12-28',
 '2020-12-29',
 '2020-12-30',
 '2020-12-31',
 '2021-01-01',
 '2021-01-02',
 '2021-01-03',
 '2021-01-04',
 '2021-01-05',
 '2021-01-06',
 '2021-01-07',
 '2021-01-08',
 '2021-01-09',
 '2021-01-10',
 '2021-01-11',
 '2021-01-12',
 '2021-01-13',
 '2021-01-14',
 '2021-01-15',
 '2021-01-16',
 '2021-01-17',
 '2021-01-18',
 '2021-01-19',
 '2021-01-20',
 '2021-01-21',
 '2021-01-22',
 '2021-01-23',
 '2021-01-24',
 '2021-01-25',
 '2021-01-26',
 '2021-01-27',
 '2021-01-28',
 '2021-01-29',
 '2021-01-30',
 '2021-01-31',
 '2021-02-01',
 '2021-02-02',
 '2021-02-03',
 '2021-02-04',
 '2021-02-05',
 '2021-02-06',
 '2021-02-07',
 '2021-02-08',
 '2021-02-09',
 '2021-02-10',
 '2021-02-11',
 '2021-02-12',
 '2021-02-13',
 '2021-02-14',
 '2021-02-15',
 '2021-02-16',
 '2021-02-17',
 '2021-02-18',
 '2021-02-19',
 '2021-02-20',
 '2021-02-21',
 '2021-02-22',
 '2021-02-23',
 '2021-02-24',
 '2021-02-25',
 '2021-02-26',
 '2021-02-

In [14]:
dates_to_scrape = set(dates) - set(current_spread_df['game_date'].unique().tolist()) 

In [15]:
# Get Spreads

gm_date = []
away_teams = []
home_teams = []
away_scoreboards = []
home_scoreboards = []
away_spreads = []
home_spreads = []

for date in tqdm(dates_to_scrape, desc='progress'):
    web = 'https://www.sportsbookreview.com/betting-odds/nba-basketball/pointspread/?date={}'.format(date)
    path = '../chromedriver.exe'
    driver = webdriver.Chrome(path)
    driver.get(web)
    sleep(random.randint(1,2))


    single_row_events = driver.find_elements_by_class_name('eventMarketGridContainer-3QipG')

    num_postponed_events = len(driver.find_elements_by_class_name('eventStatus-3EHqw'))

    num_listed_events = len(single_row_events)
    cutoff = num_listed_events - num_postponed_events

    for event in single_row_events[:cutoff]:

        away_team = event.find_elements_by_class_name('participantBox-3ar9Y')[0].text
        home_team = event.find_elements_by_class_name('participantBox-3ar9Y')[1].text
        away_teams.append(away_team)
        home_teams.append(home_team)
        gm_date.append(date)


        scoreboard = event.find_elements_by_class_name('scoreboard-1TXQV')

        home_score = []
        away_score = []

        for score in scoreboard:
            quarters = score.find_elements_by_class_name('scoreboardColumn-2OtpR')
            for i in range(len(quarters)):
                scores = quarters[i].text.split('\n')
                away_score.append(scores[0])
                home_score.append(scores[1])
            away_scoreboards.append(away_score)
            home_scoreboards.append(home_score)
            
            
        if len(away_scoreboards) != len(away_teams):
            num_to_add = len(away_teams) - len(away_scoreboards)
            for i in range(num_to_add):
                away_scoreboards.append([])
                home_scoreboards.append([])
            
        
        spreads = event.find_elements_by_class_name('pointer-2j4Dk')
        away_lines = []
        home_lines = []
        for i in range(len(spreads)):    
            if i % 2 == 0:
                away_lines.append(spreads[i].text)
            else:
                home_lines.append(spreads[i].text)
        away_spreads.append(away_lines)
        home_spreads.append(home_lines)
        
        if len(away_spreads) != len(away_teams):
            num_to_add = len(away_teams) - len(away_spreads)
            for i in range(num_to_add):
                away_scoreboards.append([])
                home_scoreboards.append([])
            
        
    driver.quit()

spreads_to_add_df = pd.DataFrame({'away_team':away_teams,
                  'home_team':home_teams,
                   'game_date':gm_date,
                  'away_scoreboard':away_scoreboards,
                  'home_scoreboard':home_scoreboards,
                  'away_spread':away_spreads,
                  'home_spread':home_spreads})

spreads_to_add_df


progress: 100%|████████████████████████████████████████████████████████████████████████| 69/69 [10:11<00:00,  8.86s/it]


,away_team,home_team,game_date,away_scoreboard,home_scoreboard,away_spread,home_spread
0,Dallas,Washington,2021-04-03,"[24, 28, 23, 34, 109]","[19, 28, 20, 20, 87]","[-6-110, -5½-115, -6-115, -6-108]","[+6-110, +5½-105, +6-105, +6-108]"
1,Cleveland,Miami,2021-04-03,"[27, 27, 19, 28, 101]","[36, 31, 17, 31, 115]","[+12-107, +12½-110, +12½-115, +12-108]","[-12-113, -12½-110, -12½-105, -12-108]"
2,Minnesota,Philadelphia,2021-04-03,"[29, 31, 23, 30, 113]","[26, 38, 33, 25, 122]","[+13½-105, +14-110, +14-110, +14-108]","[-13½-115, -14-110, -14-110, -14-108]"
3,New York,Detroit,2021-04-03,"[41, 19, 28, 37, 125]","[15, 26, 20, 20, 81]","[-2½-109, -3-110, -2½-110, -2½-108]","[+2½-111, +3-110, +2½-110, +2½-108]"
4,Indiana,San Antonio,2021-04-03,"[32, 35, 31, 28, 13, 139]","[33, 26, 38, 29, 7, 133]","[+6½-110, +7-110, +7½-115, +7½-108]","[-6½-110, -7-110, -7½-105, -7½-108]"
...,...,...,...,...,...,...,...
413,Orlando,Atlanta,2021-04-20,"[24, 24, 24, 24, 96]","[27, 38, 15, 32, 112]","[+12½-117, +12-110, +12½-110, +12-108]","[-12½-103, -12-110, -12½-110, -12-108]"
414,Brooklyn,New Orleans,2021-04-20,"[26, 31, 36, 41, 134]","[29, 30, 33, 37, 129]","[+3-110, +2½-102, +3-110, +3-108]","[-3-110, -2½-118, -3-110, -3-108]"
415,Charlotte,New York,2021-04-20,"[33, 33, 16, 15, 97]","[30, 30, 31, 18, 109]","[+5-110, +4½-105, +5-105, +5-108]","[-5-110, -4½-115, -5-115, -5-108]"
416,Minnesota,Sacramento,2021-04-20,"[39, 35, 25, 35, 134]","[43, 31, 29, 17, 120]","[+3-105, +3½-111, +3½-110, +3½-108]","[-3-115, -3½-109, -3½-110, -3½-108]"


In [18]:
for col in spreads_to_add_df.columns[3:]:
    spreads_to_add_df[col] = spreads_to_add_df[col].astype(str)
    spreads_to_add_df[col] = spreads_to_add_df[col].str.replace("[", "")
    spreads_to_add_df[col] = spreads_to_add_df[col].str.replace("]", "")
    spreads_to_add_df[col] = spreads_to_add_df[col].str.strip()

spreads_to_add_df

,away_team,home_team,game_date,away_scoreboard,home_scoreboard,away_spread,home_spread
0,Dallas,Washington,2021-04-03,"'24', '28', '23', '34', '109'","'19', '28', '20', '20', '87'","'-6-110', '-5½-115', '-6-115', '-6-108'","'+6-110', '+5½-105', '+6-105', '+6-108'"
1,Cleveland,Miami,2021-04-03,"'27', '27', '19', '28', '101'","'36', '31', '17', '31', '115'","'+12-107', '+12½-110', '+12½-115', '+12-108'","'-12-113', '-12½-110', '-12½-105', '-12-108'"
2,Minnesota,Philadelphia,2021-04-03,"'29', '31', '23', '30', '113'","'26', '38', '33', '25', '122'","'+13½-105', '+14-110', '+14-110', '+14-108'","'-13½-115', '-14-110', '-14-110', '-14-108'"
3,New York,Detroit,2021-04-03,"'41', '19', '28', '37', '125'","'15', '26', '20', '20', '81'","'-2½-109', '-3-110', '-2½-110', '-2½-108'","'+2½-111', '+3-110', '+2½-110', '+2½-108'"
4,Indiana,San Antonio,2021-04-03,"'32', '35', '31', '28', '13', '139'","'33', '26', '38', '29', '7', '133'","'+6½-110', '+7-110', '+7½-115', '+7½-108'","'-6½-110', '-7-110', '-7½-105', '-7½-108'"
...,...,...,...,...,...,...,...
413,Orlando,Atlanta,2021-04-20,"'24', '24', '24', '24', '96'","'27', '38', '15', '32', '112'","'+12½-117', '+12-110', '+12½-110', '+12-108'","'-12½-103', '-12-110', '-12½-110', '-12-108'"
414,Brooklyn,New Orleans,2021-04-20,"'26', '31', '36', '41', '134'","'29', '30', '33', '37', '129'","'+3-110', '+2½-102', '+3-110', '+3-108'","'-3-110', '-2½-118', '-3-110', '-3-108'"
415,Charlotte,New York,2021-04-20,"'33', '33', '16', '15', '97'","'30', '30', '31', '18', '109'","'+5-110', '+4½-105', '+5-105', '+5-108'","'-5-110', '-4½-115', '-5-115', '-5-108'"
416,Minnesota,Sacramento,2021-04-20,"'39', '35', '25', '35', '134'","'43', '31', '29', '17', '120'","'+3-105', '+3½-111', '+3½-110', '+3½-108'","'-3-115', '-3½-109', '-3½-110', '-3½-108'"


In [20]:
current_spread_df = pd.read_csv('../data/all_spreads_sbr.csv')

current_spread_df
# current_spread_df.loc[current_spread_df['away_spread'] == '[]']

new_current_spreads = pd.concat([current_spread_df, spreads_to_add_df])
new_current_spreads = new_current_spreads.drop_duplicates(subset=['away_team', 'home_team', 'game_date'], keep='last')
new_current_spreads['game_date'] = pd.to_datetime(new_current_spreads['game_date'])
new_current_spreads.sort_values('game_date')
new_current_spreads.to_csv("../data/all_spreads_sbr.csv", index=False)

In [108]:
updated_spreads_df.to_csv("../data/all_spreads_sbr.csv", index=False)